# PyTorchによるニューラルネットワーク

pytorchをインストールします．
CUDAのバージョンは8.0，pytorchのバージョンは0.4.0です．

pytorchに必要なパッケージ類をインポートします

In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [15]:
torch.cuda.is_available()

True

In [ ]:
input_size    = 784   
hidden_size   = 500   
num_classes   = 10 
num_epochs    = 5  
batch_size    = 100
learning_rate = 1e-3

In [6]:
train_dataset = dsets.MNIST(root='./data', train=True,  transform=transforms.ToTensor(),   download=True)
test_dataset = dsets.MNIST(root='./data', train=False,  transform=transforms.ToTensor())

Processing...
Done!


In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net, self).__init__()                  
        self.fc1 = nn.Linear(input_size, hidden_size)  
        self.relu = nn.ReLU()                          
        self.fc2 = nn.Linear(hidden_size, num_classes) 
    
    def forward(self, x):                            
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out


In [ ]:
net = Net(input_size, hidden_size, num_classes)

In [ ]:
use_cuda = True
if use_cuda and torch.cuda.is_available():
    net.cuda()

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

In [13]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, 28*28))    
        labels = Variable(labels)
        
        if use_cuda and torch.cuda.is_available():
            images = images.cuda()
            labels = labels.cuda()
        
        optimizer.zero_grad()               
        outputs = net(images)              
        loss = criterion(outputs, labels)
        loss.backward()                           
        optimizer.step()                          
        
        if (i+1) % 100 == 0: 
            print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Epoch [1/5], Step [100/600], Loss: 0.3437
Epoch [1/5], Step [200/600], Loss: 0.2466
Epoch [1/5], Step [300/600], Loss: 0.2327
Epoch [1/5], Step [400/600], Loss: 0.1445
Epoch [1/5], Step [500/600], Loss: 0.1672
Epoch [1/5], Step [600/600], Loss: 0.2163
Epoch [2/5], Step [100/600], Loss: 0.1865
Epoch [2/5], Step [200/600], Loss: 0.1227
Epoch [2/5], Step [300/600], Loss: 0.1054
Epoch [2/5], Step [400/600], Loss: 0.1351
Epoch [2/5], Step [500/600], Loss: 0.0858
Epoch [2/5], Step [600/600], Loss: 0.1538
Epoch [3/5], Step [100/600], Loss: 0.0468
Epoch [3/5], Step [200/600], Loss: 0.0767
Epoch [3/5], Step [300/600], Loss: 0.0357
Epoch [3/5], Step [400/600], Loss: 0.0680
Epoch [3/5], Step [500/600], Loss: 0.0620
Epoch [3/5], Step [600/600], Loss: 0.0751
Epoch [4/5], Step [100/600], Loss: 0.0384
Epoch [4/5], Step [200/600], Loss: 0.1384
Epoch [4/5], Step [300/600], Loss: 0.0482
Epoch [4/5], Step [400/600], Loss: 0.0598
Epoch [4/5], Step [500/600], Loss: 0.0241
Epoch [4/5], Step [600/600], Loss:

In [14]:
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, 28*28))
    
    if use_cuda and torch.cuda.is_available():
        images = images.cuda()
        labels = labels.cuda()
    
    
    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1) 
    total += labels.size(0)     
    correct += (predicted == labels).sum()    
    
print('Accuracy of the network on the 10K test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 10K test images: 98 %


In [ ]:
torch.save(net.state_dict(), 'fnn_model.pkl')